In [1]:
# Import the necessary libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2 
import scipy.io as sio
import scipy.ndimage
import glob
import csv

In [12]:
input_path = '/home/diya/Projects/flow_super_resolution/dataset/hit_data/flow_sr_ns2d/data_HR/matlab_data/' 
output_path = '/home/diya/Projects/flow_super_resolution/outputs/'

In [6]:
# Load high-resolution data generated from MATLAB code
hr_files = sorted(glob.glob(input_path))

high_res_data = []
for file in hr_files:
    mat_data = sio.loadmat(file)
    # Extract vorticity field 'omg' as mentioned in the paper
    high_res_data.append(mat_data['omg'])
    
high_res_data = np.array(high_res_data)  # Shape: [n_samples, 128, 128]
print(high_res_data.shape)

(256, 128, 128)


In [7]:
def average_downsample(data, target_size=(16, 16)):
    n_samples = data.shape[0]
    low_res_data = np.zeros((n_samples, target_size[0], target_size[1]))
    
    for i in range(n_samples):
        # Reshape to perform average pooling
        h, w = data[i].shape
        pool_size = (h // target_size[0], w // target_size[1])
        reshaped = data[i].reshape(target_size[0], pool_size[0], 
                                  target_size[1], pool_size[1])
        low_res_data[i] = reshaped.mean(axis=(1, 3))
    
    return low_res_data

# Generate low-resolution data
low_res_data_16x16 = average_downsample(high_res_data, target_size=(16,16))

In [8]:
from sklearn.model_selection import train_test_split

# First split: Train (80%) and Temp (20%)
X_train, X_temp, y_train, y_temp = train_test_split(
    low_res_data_16x16, high_res_data, test_size=0.2, random_state=42
)

# Second split: Validation (10%) and Test (10%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# Check the sizes
print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)}")


Train: 204, Validation: 26, Test: 26


#### Funciton for bicubic upsampling

In [9]:
def bicubic_upsample(data, target_size=(128, 128)):
    upsampled_data = np.zeros((data.shape[0], target_size[0], target_size[1]))
    
    for i in range(data.shape[0]):
        upsampled_data[i] = scipy.ndimage.zoom(data[i], 
                                               (target_size[0]/data.shape[1], target_size[1]/data.shape[2]), 
                                               order=3)  # Bicubic interpolation
    
    return upsampled_data

#### Apply bicubic upsampling to the LR data

In [10]:
# Upsample from 8x8 to 128x128 using bicubic interpolation
upsampled_train_16x16 = bicubic_upsample(X_train, target_size=(128, 128))
upsampled_val_16x16 = bicubic_upsample(X_val, target_size=(128, 128))
upsampled_test_16x16 = bicubic_upsample(X_test, target_size = (128, 128))

# Check shapes
print(f"Bicubic Upsampled Training Data: {upsampled_train_16x16.shape}")
print(f"Bicubic Upsampled Validation Data: {upsampled_val_16x16.shape}")
print(f"Bicubic Upsampled Test Data: {upsampled_test_16x16.shape}")


Bicubic Upsampled Training Data: (204, 128, 128)
Bicubic Upsampled Validation Data: (26, 128, 128)
Bicubic Upsampled Test Data: (26, 128, 128)


#### Save the upsampled data

In [11]:
# Define save paths
bicubic_save_dir = output_path
os.makedirs(bicubic_save_dir, exist_ok=True)

# Save the data
np.save(bicubic_save_dir + 'train.npy', upsampled_train_16x16)
np.save(bicubic_save_dir + 'val.npy', upsampled_val_16x16)

print(f"Saved upsampled datasets in {bicubic_save_dir}")


Saved upsampled datasets in /home/diya/Projects/flow_super_resolution/outputs/bicubic_inter_results/


#### Visualize the upsampled data

In [12]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

def calculate_metrics(prediction, target):
    mse = np.mean((prediction - target) ** 2)
    rmse = np.sqrt(mse)
    target_range = np.max(target) - np.min(target)
    nrmse = rmse / target_range if target_range != 0 else rmse
    data_range = target_range
    psnr = 20 * np.log10(data_range / rmse) if rmse > 0 else float('inf')

    return {"mse": mse, "rmse": rmse, "nrmse": nrmse, "psnr": psnr}

def visualize_results(low_res_samples, bilinear_samples, bicubic_samples, 
                                fno_outputs, high_res_samples, num_samples=3, dataset_type="Test", save_dir=None, save=True):
    os.makedirs(save_dir, exist_ok=True)
    indices = np.random.choice(len(low_res_samples), num_samples, replace=False)

    for i, idx in enumerate(indices):
        fig, axes = plt.subplots(2, 3, figsize=(16, 10))
        fig.suptitle(f"{dataset_type} Sample {i+1} - Super-Resolution Comparison", fontsize=18)

        titles = [
            r"$u(\Omega_L, t)$", 
            r"$BL(u(\Omega_L, t))$", 
            r"$BC(u(\Omega_L, t))$", 
            r"$u(\Omega_H, t)$", 
            r"$f_\theta(u(\Omega_L, t))$"
        ]

        images = [
            low_res_samples[idx], 
            bilinear_samples[idx], 
            bicubic_samples[idx],
            high_res_samples[idx], 
            fno_outputs[idx]
        ]

        for ax, title, img in zip(axes.flat[:5], titles, images):
            im = ax.imshow(img, cmap='coolwarm')
            ax.set_title(title)
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            ax.set_xticks([])
            ax.set_yticks([])

        # Energy Spectrum (dummy example)
        ax_spec = axes[1, 2]
        k = np.arange(1, 100)
        ax_spec.loglog(k, 1/k, label=r"$u(\Omega_H, t)$")
        ax_spec.loglog(k, 1/(k**1.2), label=r"$f_\theta(u(\Omega_L, t))$")
        ax_spec.loglog(k, 1/(k**1.4), label=r"$BC(u(\Omega_L, t))$")
        ax_spec.loglog(k, 1/(k**1.6), label=r"$BL(u(\Omega_L, t))$")
        ax_spec.set_title("Energy Spectrum")
        ax_spec.set_xlabel(r"$|\mathbf{k}|$")
        ax_spec.set_ylabel(r"$E(\mathbf{k})$")
        ax_spec.legend()
        ax_spec.grid(True, which="both", ls="--", lw=0.5)

        plt.tight_layout(rect=[0, 0, 1, 0.95])

        if save:
            plt.savefig(os.path.join(save_dir, f"{dataset_type.lower()}_composite_sample_{i+1}.png"), dpi=150)
            plt.close(fig)
        else:
            plt.show()

    print(f"Saved {num_samples} composite visualizations to {save_dir}")


In [23]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def calculate_metrics(prediction, target):
    mse = np.mean((prediction - target) ** 2)
    rmse = np.sqrt(mse)
    target_range = np.max(target) - np.min(target)
    nrmse = rmse / target_range if target_range != 0 else rmse
    psnr_value = 20 * np.log10(target_range / rmse) if rmse > 0 else float('inf')
    return {"mse": mse, "rmse": rmse, "nrmse": nrmse, "psnr": psnr_value}


def visualize_and_save_samples(
    low_res_samples,
    high_res_samples,
    upsampled_samples,
    num_samples=5,
    dataset_type="Training",
    save_dir=None
):
    if save_dir is None:
        raise ValueError("save_dir must be specified.")
    os.makedirs(save_dir, exist_ok=True)

    # LaTeX + serif font styling
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "axes.titlesize": 11,
        "axes.labelsize": 11,
        "xtick.labelsize": 11,
        "ytick.labelsize": 11
    })

    x = y = np.linspace(0, 1, 128)
    X, Y = np.meshgrid(x, y)
    indices = np.random.choice(len(low_res_samples), num_samples, replace=False)
    all_metrics = []

    for i, idx in enumerate(indices):
        lr_img = low_res_samples[idx]
        hr_img = high_res_samples[idx]
        pred_img = upsampled_samples[idx]
        error_img = np.abs(pred_img - hr_img)
        metrics = calculate_metrics(pred_img, hr_img)
        all_metrics.append(metrics)

        fig, axes = plt.subplots(2, 2, figsize=(6, 6), constrained_layout=True)
        images = [
            (np.kron(lr_img, np.ones((8, 8))), r'$(a)$ Low Resolution'),
            (hr_img, r'$(b)$ Ground Truth'),
            (pred_img, r'$(c)$ Prediction'),
            (error_img, r'$(d)$ Error Map')
        ]

        for j, (ax, (img, title)) in enumerate(zip(axes.flat, images)):
            ax.contourf(X, Y, img, levels=50, cmap=cm.RdBu_r)
            ax.set_title(title, pad=6)
            ax.set_aspect('equal')
            ax.set_xticks([0, 0.5, 1])
            ax.set_yticks([0, 0.5, 1])
            ax.set_xticklabels([r'$0$', r'$0.5$', r'$1$'])
            ax.set_yticklabels([r'$0$', r'$0.5$', r'$1$'])

            # Add x/y labels only on the appropriate sides
            if j in [2, 3]:
                ax.set_xlabel(r'$x$')
            if j in [0, 2]:
                ax.set_ylabel(r'$y$')

        fig.suptitle(
            rf"{i+1} -- RMSE: {metrics['rmse']:.4f}, "
            rf"PSNR: {metrics['psnr']:.2f} dB",
            fontsize=12
        )

        plt.savefig(os.path.join(save_dir, f"{dataset_type.lower()}_sample_{i+1}.png"), dpi=300, bbox_inches='tight')
        plt.close()

    # Save Average Metrics
    avg_metrics = {
        "mse": np.mean([m["mse"] for m in all_metrics]),
        "rmse": np.mean([m["rmse"] for m in all_metrics]),
        "nrmse": np.mean([m["nrmse"] for m in all_metrics]),
        "psnr": np.mean([m["psnr"] for m in all_metrics])
    }

    csv_path = os.path.join(save_dir, "average_metrics.csv")
    file_exists = os.path.isfile(csv_path)
    with open(csv_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["dataset_type", "mse", "rmse", "nrmse", "psnr"])
        if not file_exists:
            writer.writeheader()
        writer.writerow({"dataset_type": dataset_type, **avg_metrics})

    print(f"\n{dataset_type} Set Average Metrics:")
    print(f"MSE: {avg_metrics['mse']:.4f}")
    print(f"RMSE: {avg_metrics['rmse']:.4f}")
    print(f"NRMSE: {avg_metrics['nrmse']:.4f}")
    print(f"PSNR: {avg_metrics['psnr']:.2f} dB")
    print(f"Visualization completed for {num_samples} samples.")


In [24]:
# Visualize bicubic upsampled results
save_dir = output_path
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [26]:
visualize_and_save_samples(X_train, y_train, upsampled_samples=upsampled_train_16x16, num_samples=len(X_train), dataset_type="Training", save_dir=save_dir+'bicubic_train')
visualize_and_save_samples(X_val, y_val, upsampled_samples=upsampled_val_16x16, num_samples=len(X_val), dataset_type="Validation", save_dir=save_dir+'bicubic_val')
visualize_and_save_samples(X_test, y_test, upsampled_samples=upsampled_test_16x16, num_samples=len(X_test), dataset_type="Test", save_dir=save_dir+'bicubic_test')


Training Set Average Metrics:
MSE: 103.4405
RMSE: 10.0246
NRMSE: 0.0652
PSNR: 23.75 dB
Visualization completed for 204 samples.

Validation Set Average Metrics:
MSE: 103.8150
RMSE: 10.0334
NRMSE: 0.0650
PSNR: 23.77 dB
Visualization completed for 26 samples.

Test Set Average Metrics:
MSE: 98.4354
RMSE: 9.7432
NRMSE: 0.0639
PSNR: 23.93 dB
Visualization completed for 26 samples.
